## Get data from Finam
### 15.04.2019

In [2]:
# сonnect the necessary packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import datetime as dt
from urllib.request import urlopen

## Helpfull functions

In [3]:
# convert Date from Finam
def conv_finam(df):
    df.columns = ['Ticer', 'Per', 'Date', 'Time', 'Close']
    for i in list(df.index):
        temp = str(df.iloc[i, 2])
        temp = temp[6:8] + '.' + temp[4:6] + '.' + temp[0:4] 
        df.iloc[i, 2] = temp
        df.iloc[i, 2] = pd.to_datetime(df.iloc[i, 2], format='%d.%m.%Y')
        df.iloc[i, 2] = df.iloc[i, 2].date()
    

          
# dowload from Finam
def finam(names, name, code, date_from, date_to):
    day_from = date_from[:2]; mon_from = date_from[3:5]; year_from = date_from[6:10]
    day_to = date_to[:2]; mon_to = date_to[3:5]; year_to = date_to[6:10]
    workdate_from = year_from[2:] + mon_from + day_from
    workdate_to = year_to[2:] + mon_to + day_to
    work = dict()
    means = []
    vols = []
    var = []
    for i in range(len(name)):
        N = name[i]
        C = str(code[i])
        url = str('http://export.finam.ru/' + N + '_' + workdate_from + '_' +  workdate_to + '.csv' +  
          '?market=1&em=' + C + '&code=' + N + '&apply=0&df=' + str(int(day_from)) + '&mf=' + 
          str(int(mon_from) - 1) + '&yf=' + year_from + '&from=' + date_from + '&dt=' + 
          str(int(day_to)) + '&mt=' + str(int(mon_to) - 1) + '&yt=' + year_to + '&to=' + 
          date_to + '&p=8&f=' + N + '_' + workdate_from + '_' +  workdate_to + '&e=.csv&cn=' + N + 
          '&dtf=1&tmf=1&MSOR=1&mstime=on&mstimever=1&sep=1&sep2=1&datf=4&at=1')
        df = pd.read_csv(url)
        conv_finam(df)
        df.index = df['Date']
        df = pd.DataFrame(df.loc[:, 'Close'])
        df.columns = [N]
        if i == 0:
            main = df
        else:
            main = pd.merge(main, df, left_index=True, right_index=True)
            main = pd.concat([main, df])
    main.columns = names
    return main

In [4]:
# MOEXBC
names = ['ALRS', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'LKOH', 'MGNT', 'MTSS', 'NVTK', 'ROSN', 'SBER', 'SNGS', 'TATN', 'VTBR', 'YNDX']
# All ticker from finam
ticker = pd.read_csv('https://raw.githubusercontent.com/Aurilino/Diploma/master/MAIN_DATA/finam_list.csv', sep=';')
# used ticker
ticker = ticker[ticker['Tiker'].isin(names)]
name = list(ticker['Tiker'])
code = list(ticker['code'])

In [6]:
date_from = '01.01.2014'
date_to = '01.01.2019'
day_from = date_from[:2]; mon_from = date_from[3:5]; year_from = date_from[6:10]
day_to = date_to[:2]; mon_to = date_to[3:5]; year_to = date_to[6:10]
workdate_from = year_from[2:] + mon_from + day_from
workdate_to = year_to[2:] + mon_to + day_to
work = dict()
means = []
vols = []
var = []
for i in range(len(name)):
    N = name[i]
    C = str(code[i])
    url = str('http://export.finam.ru/' + N + '_' + workdate_from + '_' +  workdate_to + '.csv' +  
          '?market=1&em=' + C + '&code=' + N + '&apply=0&df=' + str(int(day_from)) + '&mf=' + 
          str(int(mon_from) - 1) + '&yf=' + year_from + '&from=' + date_from + '&dt=' + 
          str(int(day_to)) + '&mt=' + str(int(mon_to) - 1) + '&yt=' + year_to + '&to=' + 
          date_to + '&p=8&f=' + N + '_' + workdate_from + '_' +  workdate_to + '&e=.csv&cn=' + N + 
          '&dtf=1&tmf=1&MSOR=1&mstime=on&mstimever=1&sep=1&sep2=1&datf=4&at=1')
    df = pd.read_csv(url)
    conv_finam(df)
    chart = [np.log(list(df['Close'])[j]) - np.log(list(df['Close'])[j-1]) for j in range(1, len(list(df['Date'])))]
    means.append(np.mean(chart))
    vols.append(np.std(chart))
    var.append(-np.percentile(sorted(chart), 5))
    df.index = df['Date']
    df = pd.DataFrame(df.loc[:, 'Close'])
    df.columns = [N]
    if i == 0:
        main = df
    else:
        main = pd.merge(main, df, left_index=True, right_index=True)
    print(main.columns)
main.columns = names
summary = pd.DataFrame(dict(Ticker = names, Return = means, Risk = vols, VaR = var))

Index(['ALRS'], dtype='object')
Index(['ALRS', 'CHMF'], dtype='object')
Index(['ALRS', 'CHMF', 'FIVE'], dtype='object')
Index(['ALRS', 'CHMF', 'FIVE', 'GAZP'], dtype='object')
Index(['ALRS', 'CHMF', 'FIVE', 'GAZP', 'GMKN'], dtype='object')
Index(['ALRS', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'LKOH'], dtype='object')
Index(['ALRS', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'LKOH', 'MGNT'], dtype='object')
Index(['ALRS', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'LKOH', 'MGNT', 'MTSS'], dtype='object')
Index(['ALRS', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'LKOH', 'MGNT', 'MTSS', 'NVTK'], dtype='object')
Index(['ALRS', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'LKOH', 'MGNT', 'MTSS', 'NVTK',
       'ROSN'],
      dtype='object')
Index(['ALRS', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'LKOH', 'MGNT', 'MTSS', 'NVTK',
       'ROSN', 'SBER'],
      dtype='object')
Index(['ALRS', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'LKOH', 'MGNT', 'MTSS', 'NVTK',
       'ROSN', 'SBER', 'SNGS'],
      dtype='object')
Index(['ALRS', 'CHMF', 'FIVE', 'GAZP', 'GMKN', 'L

,Ticker,Return,Risk,VaR
0,ALRS,0.000855,0.020366,0.031990
1,CHMF,0.000890,0.018758,0.027513
2,FIVE,-0.000857,0.023666,0.039931
3,GAZP,0.000100,0.015407,0.022932
4,GMKN,0.000719,0.018542,0.026908
